# Data Cleaning

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn

In [2]:
data=pd.read_csv("Data.csv")

C:\Users\liza_\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Checking zero variance columns

In [3]:
def count_unique_values(data):
    
    unique_values=pd.DataFrame(data=data.nunique(), columns=["Number of unique values"])
    unique_values.sort_values(by=["Number of unique values"], ascending=False, inplace=True)
    
    return unique_values

In [4]:
unique_values=count_unique_values(data)
unique_values

,Number of unique values
raw_row_number,905070
location,313001
lat,62785
lng,52725
date,3468
time,1440
subject_age,91
raw_search_vehicle_description,36
raw_result_of_contact_description,27
reason_for_stop,26


In [5]:
def delete_unique_valued_columns(data, unique_values):
    zero_variance_columns=unique_values.where(unique_values==1).dropna().index
    data.drop(zero_variance_columns, axis=1, inplace=True)

In [6]:
delete_unique_valued_columns(data, unique_values)

## Looking for dupliclate data

In [7]:
duplicated_rows=data.duplicated()

In [8]:
#Show the number of duplicates
duplicated_rows.sum()

0

In [9]:
#Duplicate rows are shown
data[duplicated_rows]

,raw_row_number,date,time,location,lat,lng,district,subject_age,subject_race,subject_sex,...,citation_issued,warning_issued,outcome,contraband_found,search_conducted,search_vehicle,search_basis,reason_for_stop,raw_search_vehicle_description,raw_result_of_contact_description


In [10]:
#Duplicares are deleted
data.drop_duplicates(inplace=True)

## Manipulating missing values

In [11]:
def count_missing_values(data):
    
    missing_values=pd.DataFrame(data=data.isnull().sum(), columns=["Number of missing values"])
    missing_values["Percent"]=100*missing_values["Number of missing values"]/data.shape[0]
    missing_values.sort_values(by=["Number of missing values"], ascending=False, inplace=True)
    
    return missing_values

In [12]:
count_missing_values(data)

,Number of missing values,Percent
contraband_found,851689,94.102003
search_basis,851689,94.102003
subject_age,58888,6.506458
district,52187,5.766073
outcome,15681,1.732573
reason_for_stop,2212,0.244401
lat,1697,0.187499
lng,1697,0.187499
location,43,0.004751
time,35,0.003867


### Backfill imputation

In [13]:
#All missing values are filled using near rows

filling_method='bfill' #ALl the methods: ‘backfill’, ‘bfill’, ‘pad’, ‘ffill’

data.fillna(method=filling_method, axis=0, inplace=True)
data.fillna(0, inplace=True)

### KNN imputation

In [14]:
from sklearn.impute import KNNImputer
data=count_missing_values(data)
n_neighbors=3
imputer=KNNImputer(n_neighbors=n_neighbors)
imputer.fit(data)
data=imputer.transform(data)